In [ ]:
google_colab = True
if google_colab:
    from google.colab import drive
    import os
    drive.mount('/content/drive')
    cwd="/content/drive/MyDrive/NewsTrading/trading_bot"
    %cd /content/drive/MyDrive/NewsTrading/trading_bot
    %pip install -r requirements_clean.txt
    os.environ["TRADING_BOT_CONFIG_PATH"] = "src/config_gcs.yaml"

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from transformers import BertTokenizerFast
from src.config import config, MODEL_CONFIG
import os
from torch import Tensor
from src.utils.time import timing
from src.preprocessing.embedding import get_text_and_labels, embed_inputs, get_encoding

In [ ]:
MAX_ENCODING_LENGTH = 512
DATASET_PATH = config.data.benzinga.cleaned

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(MODEL_CONFIG.transformer_hugface_id)

In [ ]:
dataset = pd.read_parquet(DATASET_PATH)

In [ ]:
# Dummy column
dataset["text_length"] = dataset["parsed_body"].map(lambda x: len(x))

In [ ]:
texts, labels = get_text_and_labels(dat=dataset, 
                                    text_col="parsed_body", 
                                    label_col="text_length")
input_ids, masks = embed_inputs(texts, tokenizer)

In [ ]:
input_ids = pd.DataFrame(data=Tensor.numpy(input_ids), index=dataset.index)
masks = pd.DataFrame(data=Tensor.numpy(masks), index=dataset.index)

In [ ]:
input_ids.columns = [str(x) for x in input_ids.columns]
masks.columns = [str(x) for x in masks.columns]

In [ ]:
input_ids.to_parquet(config.data.benzinga.input_ids)
masks.to_parquet(config.data.benzinga.masks)